# Event Prediction
##### Predicting orbital events is essential for mission planning and operations. This notebook demonstrates how to predict these events based on numerical extrapolation. Using spacetower, you will learn how to create and configure orbital models, set up event prediction parameters, and visualize the results. This includes predicting the visibility of ground stations, detecting eclipses, and identifying node crossing events, ensuring comprehensive coverage of the satellite's behavior throughout its orbit.

## Jupyter Notebook Help Guide
Welcome to your Jupyter Notebook! If you need assistance or more information about any function, method, or object, the contextual help window is a valuable tool.

***Important - To have access to contextual help, start by running the [Imports](#imports) section below***

![Run Notebook](https://portal.exotrail.space/images/products/notebooks/execute_workflow.jpg)


### How to Open the Contextual Help Window


1. **Use the Help Menu**:
   - Navigate to the top menu bar and click on `Help`. From the dropdown menu, select `Show Contextual Help`.
2. **Keyboard Shortcut**:
   - Press `Shift + Tab` while your cursor is over the code you want more information on. For a more detailed view, press `Shift + Tab` twice.
3. **Using the Inspector**:
   - Type `?` followed by the function or object name and run the cell. For example, `?print`.
   - To see the full documentation, use `??` instead, e.g., `??print`.


### Tips for Using the Contextual Help


- **Inline Help**: Single press `Shift + Tab` to get a brief pop-up of the docstring.
- **Expanded Help**: Double press `Shift + Tab` or click the expand icon in the pop-up to open the full documentation in the help pane.
- **Persistent Help Pane**: Use the Help menu or `Shift + Tab` twice to dock the help pane on the right side of the screen, where it can stay open as you work.
This feature can help you understand function parameters, return types, and see example usages directly in your notebook. Happy coding!

## Imports

import datetime

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

from plotly.subplots import make_subplots
from fds.models.ground_station import GroundStation
from fds.models.orbit_extrapolation.requests import EventsRequestStationVisibility, EventsRequestOrbital, \
    MeasurementsRequestGpsNmea
from fds.models.orbit_extrapolation.use_case import OrbitExtrapolation
from fds.models.orbital_state import PropagationContext, OrbitalState
from fds.models.spacecraft import Battery, SolarArray, ThrusterElectrical, SpacecraftBox
from fds.models.two_line_element import TwoLineElement

*Note*: If you are using this notebook locally, use the following methods to configure your credentials:

```python
from fds.config import set_client_id, set_client_secret
set_client_id('CLIENT_ID')
set_client_secret('CLIENT_SECRET')
```

## Create Models

### Create Orbital State
`OrbitalState` is the object used in spacetower to group information on the initial orbit and covariance, the propagation context and the spacecraft definition.  

#### TLE

The TLE provides the orbital parameters needed for accurate satellite tracking and prediction.

In [ ]:
tle = TwoLineElement(
    "1 25544U 98067A   24142.35003124  .00022843  00000-0  38371-3 0  9995",
    "2 25544  51.6390  88.3709 0003333 191.4959 306.2513 15.51667899454382"
    )

print(f"TLE date (UTC): {tle.date}")

#### Propagation Context

The `PropagationContext` object encapsulates the settings necessary to accurately propagate the satellite's orbit by accounting for gravitational and non-gravitational perturbations. Adjusting these parameters allows for fine-tuning the balance between computational efficiency and the accuracy of the simulation.

In [ ]:
propagation_context = PropagationContext(
    model_perturbations=[
        PropagationContext.Perturbation.DRAG,
        PropagationContext.Perturbation.EARTH_POTENTIAL,
        PropagationContext.Perturbation.SRP,
        PropagationContext.Perturbation.THIRD_BODY,
    ],
    model_solar_flux=150,  # SFU
    model_earth_potential_deg=30,
    model_earth_potential_ord=30,
    model_atmosphere_kind=PropagationContext.AtmosphereModel.HARRIS_PRIESTER,
    integrator_kind=PropagationContext.IntegratorKind.DORMAND_PRINCE_853,
    integrator_min_step=0.01,  # s
    integrator_max_step=100,  # s
)

#### Spacecraft

Here we are using the `SpacecraftBox` class to define our spacecraft.  
To define a simpler model, try using the `SpacecraftSphere` class where battery, solar-arrays and thrusters do not need to be defined.

In [ ]:
battery = Battery(
    depth_of_discharge=0.3,  # 0<x<1
    nominal_capacity=560,  # W
    minimum_charge_for_firing=0.9,  # 0<x<1
    initial_charge=1,  # 0<x<1
)

solar_array = SolarArray(
    kind="DEPLOYABLE_FIXED",
    initialisation_kind=SolarArray.InitialisationKind.MAXIMUM_POWER,
    efficiency=.293,  # 0<x<1
    normal_in_satellite_frame=(0, 0, -1),  # Unit vector
    maximum_power=300,  # W
    surface=0.75,  # m^2
)


electrical_thruster = ThrusterElectrical(
    isp=950,  # s
    thrust=0.005,  # N
    axis_in_satellite_frame=(-1, 0, 0),  # Unit vector
    propellant_mass=4,  # kg
    wet_mass=11,  # kg
    warm_up_duration=240,  # s
    maximum_thrust_duration=1200,  # s
    impulse=37265.27,  # Ns
    power=300,  # W
    stand_by_power=1.1,  # W
    warm_up_power=50,  # W
)

spacecraft = SpacecraftBox(
    battery=battery,
    thruster=electrical_thruster,
    solar_array=solar_array,
    platform_mass=112,  # kg
    drag_coefficient=2.2,
    length_x=.5,  # m
    length_y=.5,  # m
    length_z=.5,  # m
    max_angular_velocity=2,  # deg/s
    max_angular_acceleration=.5,  # deg/s^2
)

#### Orbital State

Initiate the `OrbitalState` object with the TLE, the propagation context and the spacecraft.

In [ ]:
orbital_state = OrbitalState.from_tle(
    tle=tle,
    propagation_context=propagation_context,
    spacecraft=spacecraft,
)

### Create Event Prediction Configuration
In this section we define the parameters for the event prediction. This includes the prediction window, ground stations, and the type of events to predict.

In [ ]:
# Set up station visibility request starting from the orbital state date 
# Try defining your own ground stations using the GroundStation class with name, latitude, longitude, altitude, and minimum elevation
station_events_request = EventsRequestStationVisibility(
    start_date=orbital_state.date,
    ground_stations=[GroundStation("Sapporo", 43.053451, 141.335871, 0., 5, )]
)

# Define the types of orbital events to be requested
orbital_events = EventsRequestOrbital(
    event_kinds=[EventsRequestOrbital.EventKind.ECLIPSE,
                 EventsRequestOrbital.EventKind.NODE],
    start_date=orbital_state.date)

### Create NMEA Measurements Request (for ground track plot)

In [ ]:
# Measurement request (for the ground track visualization)
gps_nmea_request = MeasurementsRequestGpsNmea(
    standard_deviation_altitude=1E-10,
    standard_deviation_latitude=1E-10,
    standard_deviation_longitude=1E-10,
    standard_deviation_ground_speed=1E-10,
    generation_step=60
)

## Build & Run Use Case

In [ ]:
# This example will propagate the orbit for 3 orbits since the TLE date
# *Try changing the number of orbits to propagate the orbit for a different duration*
orbits = 3

oe = OrbitExtrapolation.with_target_date(
    target_date=orbital_state.date + datetime.timedelta(seconds=orbital_state.mean_orbit.keplerian_period * orbits),
    initial_orbital_state=orbital_state,
    measurements_request=gps_nmea_request,
    orbital_events_request=orbital_events,
    station_visibility_events_request=station_events_request
)

print(f"Propagation orbit from TLE date to: {orbits} orbits from now")
print(f"Start date: {oe.initial_date}")
print(f"End date: {oe.final_date}")
print(f"Duration: {format(oe.duration / 3600., '.3f')} hours")

# Run the orbit extrapolation
oe.run()

## Results/Post-Processing

### Extract Results and Prepare Ground Track Data

In [ ]:
# Extract results from the orbit extrapolation 
res = oe.result

# Retrieve the dates and measurements (latitude and longitude) from the results
dates = np.array(res.computed_measurements[0].dates)
latitude = np.array(res.computed_measurements[0].measurements)[:, 0]
longitude = np.array(res.computed_measurements[0].measurements)[:, 1]

# Filter out measurements before the current time
now = datetime.datetime.now(datetime.UTC)
now_index = np.argmax(dates > now)
dates = dates[now_index:]
latitude = latitude[now_index:]
longitude = longitude[now_index:]

# Calculate relative times from the start date
relative_times = [(date - dates[0]).total_seconds() / 3600 for date in dates]

# Identify the closest times to station visibility start and end events
station_visibility_start = []
station_visibility_end = []
lat_closest_start, lon_closest_start, lat_closest_end, lon_closest_end = [], [], [], []
if res.station_visibility_events is not None:
    for event in res.station_visibility_events:
        station_visibility_start.append(np.argmin(np.abs(dates - event.start_date)))
        station_visibility_end.append(np.argmin(np.abs(dates - event.end_date)))

    lat_closest_start = latitude[station_visibility_start]
    lon_closest_start = longitude[station_visibility_start]
    lat_closest_end = latitude[station_visibility_end]
    lon_closest_end = longitude[station_visibility_end]

# Create a DataFrame with the longitude, latitude, and date for plotting
lon_lat_df = pd.DataFrame({'Longitude': longitude, 'Latitude': latitude, 'Date': dates})

### Plot Results

#### Orbit Track

In [ ]:

fig = go.Figure()

# Plot the orbit track
fig.add_trace(go.Scattergeo(
    lat=lon_lat_df['Latitude'],
    lon=lon_lat_df['Longitude'],
    mode='markers+lines',
    line=dict(width=1, color='black'),
    marker=dict(
        size=4,
        color='black',
    ),
    name="Orbit track",
))

# Add a point for the initial position
fig.add_trace(go.Scattergeo(
    lat=[latitude[0]],
    lon=[longitude[0]],
    mode='markers',
    marker=dict(
        size=8,
        color='red',
    ),
    name=f"Initial position at {dates[0]}",
))

# Add a point for the ground station closest to the start of the visibility
if res.station_visibility_events is not None:
    for i in range(len(station_visibility_start)):
        fig.add_trace(go.Scattergeo(
            lat=[lat_closest_start[i]],
            lon=[lon_closest_start[i]],
            mode='markers',
            marker=dict(
                size=12,
                color='green',
                symbol='triangle-up'
            ),
            name=f"Visibility start ({dates[station_visibility_start[i]]})",
        ))

    # Add a point for the ground station closest to the end of the visibility
    for i in range(len(station_visibility_end)):
        fig.add_trace(go.Scattergeo(
            lat=[lat_closest_end[i]],
            lon=[lon_closest_end[i]],
            mode='markers',
            marker=dict(
                size=12,
                color='orange',
                symbol='triangle-down'
            ),
            name=f"Visibility end ({dates[station_visibility_end[i]]})",
        ))

# Add a point for the target position
fig.add_trace(go.Scattergeo(
    lat=[station_events_request.ground_stations[0].coordinates.latitude],
    lon=[station_events_request.ground_stations[0].coordinates.longitude],
    mode='markers',
    marker=dict(
        size=8,
        color='blue',
    ),
    name=station_events_request.ground_stations[0].name,
))

fig.update_layout(
    title_text=f"Orbit track",
)
fig.show()

#### Events Timeline

In [ ]:
# Export the event data to a DataFrame 
dat = res.export_event_gantt_data()
df_timeline = pd.DataFrame(dat)

# Filter out DESCENDING and ASCENDING events
df_timeline = df_timeline[df_timeline["event"] != "DESCENDING_NODE"]
df_timeline = df_timeline[df_timeline["event"] != "ASCENDING_NODE"]

# Create the timeline plot
fig = px.timeline(df_timeline, x_start="start_date", x_end="end_date", y="event", color="event",
                  labels={'Task': 'Event'},
                  title='Events timeline',
                  hover_name="event",
                  hover_data={"ground_station_name": True}
                  )
fig.show()